## Setup

First, we need to install the necessary library and set up the Groq API client.

Make sure you have your Groq API key stored in Colab secrets under the name `GROQ_API_KEY`.

In [1]:
!pip install groq ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.0 MB/s eta 0:00:00


In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from groq import Groq
import os
from google.colab import userdata, files

# Initialize Groq client using Colab secrets
api_key = userdata.get('GROQ_API_KEY')
client = Groq(api_key=api_key)
print("API key successfully loaded from Colab secrets")

API key successfully loaded from Colab secrets


## Upload Articles

Now, upload the three text files containing your research articles.

In [3]:
# Upload files widget
print("Please upload exactly 3 text files containing research articles:")
uploaded = files.upload()

# Store file contents and filenames
file_contents = []
filenames = []

if len(uploaded) != 3:
    print("Please upload exactly 3 files")
else:
    for filename, content in uploaded.items():
        article_text = content.decode('utf-8')
        file_contents.append(article_text)
        filenames.append(filename)
        print(f"Loaded: {filename} ({len(article_text.split())} words)")

# A simple validation function (no length restriction)
def validate_article_length(article, filename):
    # No length restriction for this example
    return None

Please upload exactly 3 text files containing research articles:


Saving ChatGPT Search A Guide With Example.txt to ChatGPT Search A Guide With Example.txt
Saving Context Engineering A Guide With Ex.txt to Context Engineering A Guide With Ex.txt
Saving Small Language Models A Guide With.txt to Small Language Models A Guide With.txt
Loaded: ChatGPT Search A Guide With Example.txt (2203 words)
Loaded: Context Engineering A Guide With Ex.txt (2085 words)
Loaded: Small Language Models A Guide With.txt (2457 words)


## Define Prompting Functions

We'll define functions for zero-shot, one-shot, and few-shot prompting based on the desired summary format.

In [4]:
# --- Advanced Prompting Techniques ---

# --- 1. Chain-of-Thought (CoT) Prompting ---
# This replaces the original zero_shot_prompt
def cot_prompt(article):
    return f"""
    Please analyze the following research article step by step to create a summary.

    Article: {article}

    Think through your summary creation process:
    1. First, identify the background and introduction.
    2. Next, determine the main research objective.
    3. Then, describe the methodology used.
    4. After that, highlight the key results.
    5. Finally, summarize the main takeaways and conclusions.

    Now, provide the final summary based on this analysis, structured as:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]
    """

# --- 2. Tree-of-Thoughts (ToT) Prompting (Simplified Simulation) ---
# This replaces the original one_shot_prompt
def tot_prompt(article, example_article, example_summary):
    # The example is used to prime the model for the ToT structure
    return f"""
    Consider the research article below. To understand it thoroughly, explore different perspectives or ways to summarize it.

    Example Article: {example_article}
    Example Summary (for structure reference): {example_summary}

    Now, for the following article:
    Article: {article}

    1. Path 1: Focus on the technical methodology and results.
    2. Path 2: Focus on the implications and conclusions for the field.
    3. Path 3: Focus on the background context and research gap addressed.

    For each path, briefly outline the key points you would include in a summary.

    Then, synthesize the insights from these paths to create a final, well-rounded summary structured as:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]
    """

# --- 3. Role-Based Prompting ---
def role_based_prompt(article):
    return f"""
    You are a renowned expert in artificial intelligence research. Your task is to summarize the following technical article for a general audience of researchers new to the field.

    Article: {article}

    Provide a clear and concise summary structured as:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]
    """

# --- 4. ReAct Prompting (Reasoning + Action Simulation) ---
def react_prompt(article):
    return f"""
    You are tasked with summarizing a research article. Use a ReAct (Reasoning + Action) approach.

    Article: {article}

    Thought 1: I need to understand the core components of the article.
    Action 1: Identify the background/introduction section.
    Observation 1: [The model identifies the background/introduction]
    Thought 2: Now I need to find the research objective.
    Action 2: Locate the statement of the research goal.
    Observation 2: [The model identifies the research objective]
    Thought 3: Next, I need details on how the research was conducted.
    Action 3: Extract information about the methodology.
    Observation 3: [The model identifies the methodology]
    Thought 4: What were the main findings?
    Action 4: Find the key results presented.
    Observation 4: [The model identifies the key results]
    Thought 5: Finally, what are the conclusions and implications?
    Action 5: Summarize the main takeaways/conclusions.
    Observation 5: [The model identifies the main takeaways/conclusions]

    Based on these thoughts and observations, provide the final summary structured as:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]
    """

# --- 5. Directional Stimulus Prompting ---
def directional_stimulus_prompt(article):
    return f"""
    Focus Stimulus: Summarize concisely.
    Detail Stimulus: Include all key technical terms.
    Perspective Stimulus: From the author's viewpoint.
    Article: {article}

    Based on the stimuli above, create a summary structured as:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]
    """

# --- 6. Step-Back Prompting ---
def step_back_prompt(article):
    return f"""
    Before summarizing the specific article, first answer these general questions:
    - What is the typical structure of a research article?
    - What are the key elements to include in a summary of a research article?

    Now, applying these general principles, summarize the following article:

    Article: {article}

    Provide the summary structured as:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]
    """

# --- 7. Zero-Shot (Baseline - Original Structure) ---
# Keep the original zero-shot as one of the options for comparison
def zero_shot_prompt(article):
    return f"""
    Summarize the following research article in a single paragraph of 150 words or less. Include the following sections explicitly:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]

    Article: {article}
    """

# --- 8. One-Shot (Baseline - Original Structure) ---
# Keep the original one-shot as one of the options for comparison
def one_shot_prompt(article, example_article, example_summary):
    return f"""
    Summarize the following research article in a single paragraph of 150 words or less. Include the following sections explicitly:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]

    Example Article: {example_article}
    Example Summary: {example_summary}

    Now summarize this article, including the explicit headings for each section:
    {article}
    """

# --- 9. Few-Shot (Baseline - Original Structure) ---
# Keep the original few-shot as one of the options for comparison
def few_shot_prompt(article, examples):
    prompt = """
    Summarize the following research article in a single paragraph of 150 words or less. Include the following sections explicitly:
    Background/introduction: [Content]
    Research objective: [Content]
    Methodology: [Content]
    Key results: [Content]
    Main takeaways/conclusions: [Content]
    """
    for i, (ex_article, ex_summary) in enumerate(examples):
        prompt += f"\nExample {i+1}:\nArticle: {ex_article}\nSummary: {ex_summary}\n"
    prompt += f"\nNow summarize this article, including the explicit headings for each section:\n{article}"
    return prompt


## Define API Call Function

This function sends the prompt to the Groq API and returns the generated summary. We will use the `meta-llama/llama-4-scout-17b-16e-instruct` model as requested.

In [5]:
# Function to call Groq API
def get_summary(prompt):
    completion = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct", # Using the requested model
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_completion_tokens=200, # Keep max tokens for summary consistent
        top_p=1,
        stream=False,
        stop=None,
    )
    return completion.choices[0].message.content.strip()

# Function to validate word count of the summary
def validate_summary(summary):
    word_count = len(summary.split())
    if word_count > 150:
        return f"Word count: {word_count}"
    else:
        return f"Word count: {word_count}"

## Process Articles Sequentially

This section sets up the UI and the logic to process the uploaded articles sequentially using the defined prompting techniques and save the output.

In [6]:
# --- Cell 5: Process Articles with Advanced Prompting Techniques ---

# Create UI widgets
process_button = widgets.Button(
    description='Process Articles with Advanced Prompts',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Process all articles using various advanced prompting techniques'
)

output_area = widgets.Output()

# Display UI
display(widgets.VBox([process_button, output_area]))

# Button click handler for processing and saving
def on_process_clicked(b):
    global file_contents, filenames # Ensure we access the variables from earlier cells

    if len(file_contents) != 3:
        with output_area:
            clear_output()
            print("Please upload exactly 3 files first")
        return

    # Simple validation (though the original had no length check)
    validation_errors = []
    for i, (article, filename) in enumerate(zip(file_contents, filenames)):
        error = validate_article_length(article, filename)
        if error:
            validation_errors.append(f"File {filename}: {error}")

    if validation_errors:
        with output_area:
            clear_output()
            print("Article validation errors:")
            for error in validation_errors:
                print(error)
        return

    output_text = ""
    with output_area:
        clear_output()
        print("Starting processing of articles with advanced prompting techniques...\n")

    # Store summaries generated, potentially for use in techniques that need examples
    # Key: (article_index, technique_name) -> summary
    all_summaries = {}
    # Also store summaries in a list per article for techniques like Few-Shot that need multiple examples
    # Index corresponds to article index
    article_summaries_list = [[] for _ in range(len(file_contents))]

    # --- Define the techniques to apply ---
    # Format: (Display Name, Function, Requires Example (bool), Requires Multiple Examples (bool))
    techniques = [
        ("Zero-Shot (Baseline)", zero_shot_prompt, False, False),
        ("Chain-of-Thought", cot_prompt, False, False),
        ("Role-Based", role_based_prompt, False, False),
        ("Directional Stimulus", directional_stimulus_prompt, False, False),
        ("Step-Back Abstraction", step_back_prompt, False, False),
        ("ReAct", react_prompt, False, False),
        ("One-Shot (Baseline)", one_shot_prompt, True, False), # Requires 1 example
        ("Tree-of-Thoughts (Simplified)", tot_prompt, True, False), # Requires 1 example
        ("Few-Shot (Baseline)", few_shot_prompt, False, True), # Requires multiple examples
    ]

    # --- Process each article ---
    for i in range(len(file_contents)):
        article_content = file_contents[i]
        article_filename = filenames[i]

        output_text += "="*80 + "\n"
        output_text += f"Processing Article {i+1}: {article_filename}\n"
        output_text += "="*80 + "\n"
        with output_area:
            print("="*80)
            print(f"Processing Article {i+1}: {article_filename}")
            print("="*80)

        # --- Apply each technique to the current article ---
        for tech_name, prompt_func, needs_example, needs_multiple_examples in techniques:
            output_text += "-"*40 + f"\nTechnique: {tech_name}\n" + "-"*40 + "\n"
            with output_area:
                 print("-" * 40)
                 print(f"Technique: {tech_name}")
                 print("-" * 40)

            prompt = ""
            try:
                if needs_multiple_examples:
                    # --- Few-Shot (Baseline) ---
                    examples_for_few_shot = []
                    # Use summaries from previous articles (Articles 0 up to i-1)
                    for j in range(i):
                         # Use the Zero-Shot summary of previous articles as examples
                         key = (j, "Zero-Shot (Baseline)")
                         if key in all_summaries:
                              examples_for_few_shot.append((file_contents[j], all_summaries[key]))
                         # If Zero-Shot not available, try any summary from that article
                         # elif article_summaries_list[j]:
                         #      examples_for_few_shot.append((file_contents[j], article_summaries_list[j][0]))

                    if len(examples_for_few_shot) >= 2: # Ensure we have enough examples
                        prompt = prompt_func(article_content, examples_for_few_shot)
                        # print(f"DEBUG: Using {len(examples_for_few_shot)} examples for Few-Shot.")
                    elif len(examples_for_few_shot) == 1:
                        # Fallback to One-Shot if only one example
                        prompt = one_shot_prompt(article_content, examples_for_few_shot[0][0], examples_for_few_shot[0][1])
                        tech_name = f"{tech_name} (Fallback to One-Shot)"
                        output_text += "[Info: Only 1 example available, falling back to One-Shot structure]\n"
                        with output_area:
                            print("[Info: Only 1 example available, falling back to One-Shot structure]")
                    else:
                        # Fallback to Zero-Shot if no examples
                        prompt = zero_shot_prompt(article_content)
                        tech_name = f"{tech_name} (Fallback to Zero-Shot)"
                        output_text += "[Warning: No examples available, falling back to Zero-Shot]\n"
                        with output_area:
                            print("[Warning: No examples available, falling back to Zero-Shot]")

                elif needs_example:
                     # --- Techniques needing a single example (One-Shot, ToT) ---
                     if i > 0: # Need a previous article
                         # Use the Zero-Shot summary of the immediately preceding article
                         example_key = (i-1, "Zero-Shot (Baseline)")
                         if example_key in all_summaries:
                             example_article = file_contents[i-1]
                             example_summary = all_summaries[example_key]
                             prompt = prompt_func(article_content, example_article, example_summary)
                         else:
                             # Fallback if Zero-Shot summary of previous article isn't available
                             prompt = zero_shot_prompt(article_content)
                             tech_name = f"{tech_name} (Fallback to Zero-Shot - No Example Summary)"
                             output_text += "[Warning: Example summary not found, falling back to Zero-Shot]\n"
                             with output_area:
                                 print("[Warning: Example summary not found, falling back to Zero-Shot]")
                     else:
                         # Fallback to Zero-Shot for the first article if technique needs an example
                         prompt = zero_shot_prompt(article_content)
                         tech_name = f"{tech_name} (Fallback to Zero-Shot - First Article)"
                         output_text += "[Info: First article, no example available, falling back to Zero-Shot]\n"
                         with output_area:
                            print("[Info: First article, no example available, falling back to Zero-Shot]")

                else:
                    # --- Techniques needing only the article content (Zero-Shot, CoT, Role-Based, etc.) ---
                    prompt = prompt_func(article_content)

                # --- Get Summary from LLM ---
                summary = get_summary(prompt)

                # --- Store Summary ---
                summary_key = (i, tech_name)
                all_summaries[summary_key] = summary
                article_summaries_list[i].append(summary) # Add to list for this article

                # --- Output Results ---
                output_text += f"{tech_name} Summary:\n{summary}\n"
                output_text += f"{validate_summary(summary)}\n\n"

                with output_area:
                    print(f"{tech_name} Summary:")
                    print(summary)
                    print(validate_summary(summary))
                    print("\n")

            except Exception as e:
                error_msg = f"Error processing Article {i+1} with {tech_name}: {e}"
                output_text += f"{error_msg}\n\n"
                with output_area:
                    print(error_msg)
                    print("\n")

    # --- Final Summary Section ---
    output_text += "="*80 + "\n"
    output_text += "FINAL SUMMARY OF ALL APPROACHES FOR EACH ARTICLE\n"
    output_text += "="*80 + "\n"
    with output_area:
        print("="*80)
        print("FINAL SUMMARY OF ALL APPROACHES FOR EACH ARTICLE")
        print("="*80)

    for i in range(len(file_contents)):
        output_text += f"\n--- Article {i+1}: {filenames[i]} ---\n"
        output_text += f"Original Word Count: {len(file_contents[i].split())}\n"
        with output_area:
            print(f"\n--- Article {i+1}: {filenames[i]} ---")
            print(f"Original Word Count: {len(file_contents[i].split())}")

        # List summaries generated for this article
        for key, summary in all_summaries.items():
            article_idx, tech_name = key
            if article_idx == i:
                summary_word_count = len(summary.split())
                output_text += f"  - {tech_name}: Summary Word Count: {summary_word_count}\n"
                with output_area:
                     print(f"  - {tech_name}: Summary Word Count: {summary_word_count}")

    # --- Save output to a text file ---
    output_filename = "advanced_prompting_summaries.txt"
    try:
        with open(output_filename, "w", encoding='utf-8') as f: # Specify encoding
            f.write(output_text)
        with output_area:
            print(f"\n✅ Output successfully saved to '{output_filename}'")
    except Exception as e:
        with output_area:
            print(f"\n❌ Error saving output to file: {e}")

# Attach event handler
process_button.on_click(on_process_clicked)
